## Load/import packages

In [68]:
import json
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

from os import listdir, mkdir, rename
from os.path import splitext
from os.path import join
from skimage import io, color, exposure

%matplotlib inline

## Pre-processing
1. Apply illumination normalization (histogram equalization)
2. Pose normalization
3. Store the image in a named folder same name as video. The image should be stored under the as videoname_frameXXXX ✔
4. Data augmentation to upsample some classes

## Directories
### Aff-Wild2

In [34]:
from load_filenames import (
    AW2_cropped_aligned_dir,
    AW2_cropped_aligned_folders,
    AW2_train_FN_split,
    AW2_val_FN_split,
)

In [35]:
AW2_cropped_aligned_dir

'D:\\Aff-Wild2 Dataset\\Aff-wild2\\Images\\cropped_aligned'

In [36]:
AW2_cropped_aligned_folders

['1-30-1280x720',
 '10-60-1280x720',
 '10-60-1280x720_right',
 '100-29-1080x1920',
 '101-30-1080x1920',
 '102',
 '102-30-640x360',
 '103',
 '103-30-384x480',
 '104-17-720x480',
 '105',
 '105-30-1280x720',
 '106',
 '106-30-720x1280',
 '107',
 '107-30-640x480',
 '108',
 '108-15-640x480',
 '109-30-1280x720',
 '11-24-1920x1080',
 '110',
 '110-30-270x480',
 '111',
 '111-25-1920x1080',
 '112',
 '112-30-640x360',
 '113',
 '113-60-1280x720',
 '114',
 '114-30-1280x720',
 '115-30-1280x720',
 '116',
 '116-30-1280x720',
 '117',
 '117-25-1920x1080',
 '118',
 '118-30-640x480',
 '119',
 '119-30-848x480',
 '12-24-1920x1080',
 '120',
 '120-30-1280x720',
 '121',
 '121-24-1920x1080',
 '122',
 '122-60-1920x1080-1',
 '122-60-1920x1080-2',
 '122-60-1920x1080-3',
 '122-60-1920x1080-4',
 '122-60-1920x1080-5',
 '123',
 '123-25-1920x1080',
 '124-30-720x1280',
 '125',
 '125-25-1280x720',
 '126',
 '126-30-1080x1920',
 '127',
 '127-30-1280x720',
 '128',
 '128-24-1920x1080',
 '129',
 '129-24-1280x720',
 '13-30-1920

# Start Pre-processing

Each image has the shape (112, 112, 3)

To illumination normalize it with adaptive histogram equalization, the image is converted to grayscale and gets the shape (112, 112). Afterwards it is converted back to RGB but remains the grayscale color. It's also converted back to values ranging from 0-255

In [37]:
AW2_normalized_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Images\cropped_aligned_normalized"

***
# Code below has already run

In [40]:
def pre_processing():
    start = time.time()
    for folder in AW2_cropped_aligned_folders:
        folder_dir = join(AW2_cropped_aligned_dir, folder)
        frames_FN = listdir(folder_dir)
        print(
            f"Done! Time ran since start: {round(time.time()-start)//60 }:{round(time.time()-start)%60}"
        )
        print(folder)
        # Create a directory with the name of the video to save all the normalized frames
        try:
            mkdir(join(AW2_normalized_dir, folder))
        except FileExistsError:
            print("Directory already exists!")

        for frame_name in frames_FN:
            if frame_name == ".DS_Store":
                continue
            frame = io.imread(join(folder_dir, frame_name))
            frame_g = color.rgb2gray(frame)
            frame_g_hist = exposure.equalize_adapthist(frame_g)
            frame_rgb_hist = color.gray2rgb(frame_g_hist)

            file = join(join(AW2_normalized_dir, folder), frame_name)
            io.imsave(file, (frame_rgb_hist * 255).astype(np.uint8))

In [ ]:
pre_processing()

## Move images in their correct train, val and test set directory

In [10]:
import shutil

In [75]:
# Load Aff-Wild2 AW2_train_classes.json
with open("AW2_train_classes.json", "r") as fp:
    AW2_train_classes = json.load(fp)

# Load Aff-Wild2 AW2_val_classes.json
with open("AW2_val_classes.json", "r") as fp:
    AW2_val_classes = json.load(fp)

In [12]:
train_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets\train"
val_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets\val"
test_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets\test"

In [ ]:
# Move training files
for folder, ext in AW2_train_FN_split:
    shutil.move(join(AW2_normalized_dir, folder), train_dir)

for folder, ext in AW2_val_FN_split:
    shutil.move(join(AW2_normalized_dir, folder), val_dir)

for folder in listdir(AW2_normalized_dir):
    shutil.move(join(AW2_normalized_dir, folder), test_dir)

## Put each corresponding frame in the correct class folder for training and validiation set. 
This is necessary to create a dataset in keras

In [69]:
# Exctract image from each videofolder and move it to the corresponding class folder
train_perclass_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets_per_class\train"
label_folders = listdir(train_perclass_dir)

for folder in listdir(train_dir):
    ##e.g. returns folder: 51-30-1280x720
    
    # Gets labels from the corresponding video from the AW2_train_classes dict
    labels = AW2_train_classes.get(folder + ".txt")
    
    # Loop through each frame, and take the corresponding label for this frame from labels
    for file in listdir(join(train_dir,folder)):
        frame_n, ext = splitext(file)
        frame_n = int(frame_n)
        label = labels[(frame_n-1)]
        
        # If the label is between 0-6, create corresponding new directory
        if label in [0, 1, 2, 3, 4, 5, 6]:
            new_dir = join(train_perclass_dir, label_folders[label])
        else:
            continue
        shutil.copy(join(train_dir,join(folder,file)), new_dir)
        rename(join(new_dir, file), join(new_dir, folder + "_" + file))

In [ ]:
# Exctract image from each videofolder and move it to the corresponding class folder
val_perclass_dir = r"D:\Aff-Wild2 Dataset\Aff-wild2\Sets_per_class\val"
label_folders = listdir(val_perclass_dir)

for folder in listdir(val_dir):
    ##e.g. returns folder: 51-30-1280x720
    # Gets labels from the corresponding video from the AW2_train_classes dict
    labels = AW2_val_classes.get(folder + ".txt")
  
    # Loop through each frame, and take the corresponding label for this frame from labels
    for file in listdir(join(val_dir,folder)):
        frame_n, ext = splitext(file)
        frame_n = int(frame_n)
        label = labels[(frame_n-1)]
        
        # If the label is between 0-6, create corresponding new directory
        if label in [0, 1, 2, 3, 4, 5, 6]:
            new_dir = join(val_perclass_dir, label_folders[label])
        else:
            continue
        shutil.copy(join(val_dir,join(folder,file)), new_dir)
        rename(join(new_dir, file), join(new_dir, folder + "_" + file))

*** 